In [ ]:
from gs_quant.common import Currency, OptionType
from gs_quant.markets import PricingContext
from gs_quant.markets.portfolio import Portfolio
from gs_quant.instrument import FXOption, FXForward
from gs_quant.session import GsSession, Environment
from gs_quant import risk

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
cross = "GBPJPY"
premium_currency = Currency.GBP

In [ ]:
option = FXOption(cross, expiration_date="2y", option_type=OptionType.Call, strike_price="25d", premium_currency=premium_currency, premium_payment_date="spot")
option.resolve()
# Here premium will have resolved to equal the option value 

In [ ]:
spot = option.calc(risk.FXSpot)
spot

In [ ]:
 # The CSA terms for FX Fwds won't always be USD-OIS like for derrivs, so we force everything to USD-OIS to be consistent
PricingContext.current = PricingContext(csa_term='USD-OIS')

### Black-Scholes Detla
`FXQuotedDelta` will calculate Black-Scholes delta in % terms and is the delta metric that is used to solve if you specify the option strike in delta like above

In [ ]:
option.calc(risk.FXQuotedDelta)

### Delta PnL
The `FXDelta` measure captures the USD present value change from the move in spot rates for each USD cross (in normal convention). i.e. `PnL(USD) = dSpot * FXDelta`  
Note: USD can be changed for another currency by parameterising the measure `FXDelta(currency='EUR')`

In [ ]:
opt_fxdelta = option.calc(risk.FXDelta)
opt_fxdelta.to_frame()

### Delta as Notional of Fwd
To compute the wing delta as the notional amount of an equivalent FX Fwd there are the measures `FXCalcDelta` and `FXDeltaHedge`.   
`FXCalcDelta` returns a percentage factor of the option notional amount for the option cross.  
`FXDeltaHedge` returns notional amounts of FX Fwds on USD crosses to hedge the option.

In [ ]:
# Here this will return a number close to the 0.25 B-S delta
opt_fxcalc_notional = option.calc(risk.FXCalcDelta)
opt_fxcalc_notional

We can build the FXForward settling on spot date, with that `-1 * FXCalcDelta * notional` and check the FXDelta offsets, i.e. this is the spot trade on the option cross to hedge delta

In [ ]:
fwd = FXForward(cross, "0b", "f", notional_amount=-1*float(opt_fxcalc_notional)*option.notional_amount)

In [ ]:
fwd_risk = fwd.calc(risk.FXDelta)
fwd_risk

**Alternatively**: that hedge can be broken down into the two equivalent USDJPY and GBPUSD hedges, the `FXDeltaHedge` measure is already scaled by `-1 * option notional`

In [ ]:
opt_fxhedge = option.calc(risk.FXDeltaHedge)
opt_fxhedge

In [ ]:
usd_fwds = [FXForward(opt_fxhedge.iloc[i].mkt_asset[4:] + opt_fxhedge.iloc[i].mkt_asset[:3], "0b", "f", opt_fxhedge.iloc[i].value) for i in range(len(opt_fxhedge))]

In [ ]:
Portfolio(usd_fwds).calc(risk.FXDelta).aggregate()